In [1]:
import torch.nn.functional as F
import torch
import gc
import numpy as np
from torch import Tensor
from torch.utils.data import DataLoader, Dataset, IterableDataset
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import pandas as pd
import os

/home/raki/raki-projects/e5-finetuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

DEVICE = 'cuda:7'
BATCH_SIZE = 128

In [3]:
# https://huggingface.co/intfloat/multilingual-e5-base


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
model = model.to(DEVICE)
model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwi

In [4]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(DEVICE)

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[90.81391906738281, 72.13390350341797], [70.53540802001953, 88.7610855102539]]


In [5]:
df = pd.read_csv("data/bible.csv")
df.head()

# df_pairs = df[["text_ru", "text_lez"]].reset_index(drop=True)


,Unnamed: 0,chapter,start_verse,end_verse,text_ru,text_lez,book_name_ru,book_name_lez
0,0,1,1,1,В первый день второго месяца ( в середине весн...,"Кьвед лагьай вацран сад лагьай юкъуз, исраилви...",Числа,Кьурумда
1,1,1,45,45,"Все исроильтяне от двадцати лет и старше, годн...","Аскервиле къуллугъ авуниз виже къведай, чпин к...",Числа,Кьурумда
2,2,1,25,25,В роду Гада их было 45 650.,Гадан тайифада абурун кьадар 45650 тир.,Числа,Кьурумда
3,3,1,46,46,Всего их было 603 550.,Абурун виридан кьадар 603550 тир.,Числа,Кьурумда
4,4,1,24,24,Из потомков Гада: все мужчины двадцати лет и с...,Гадан эвледрикай: аскервиле къуллугъ авуниз ви...,Числа,Кьурумда


In [6]:

class ColumnDataset(Dataset):
    def __init__(self, df, column):
        self.df_column = df[column].to_list()

    def __len__(self):
        return len(self.df_column)

    def __getitem__(self, idx):
        return self.df_column[idx]
    

dataset = ColumnDataset(df, column="text_ru")
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)



In [7]:

embeddings = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
        batch_dict = {k: v.to(DEVICE) for k, v in batch_dict.items()}
        
        outputs = model(**batch_dict)
        batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        
        for embedding in batch_embeddings:
            embeddings.append(embedding.cpu().numpy().copy())

model = model.cpu()
del model

100%|██████████| 431/431 [00:53<00:00,  8.12it/s]


### Hard Negative Mining

In [8]:
def get_negatives(df: pd.DataFrame, embeddings, min_score, max_score, max_n = 10):
    negative_ids = []

    similarity_matrix = cosine_similarity(embeddings)

    for i in tqdm(range(len(embeddings))):
        valid_ids = [j for j in range(len(embeddings)) if i != j 
                    and min_score <= similarity_matrix[i][j] <= max_score]
        
        if valid_ids:
            # TODO: Shuffle list or sort?
            valid_ids = valid_ids[:max_n] if len(valid_ids) > max_n else None
            # negative_ids.append(np.random.choice(valid_ids))
            negative_ids.append(valid_ids)
        else:
            raise ValueError("Can't find negative example, change search range")
    
    df_with_negatives = df.copy()
    df_with_negatives["negative_id"] = negative_ids

    return df_with_negatives

In [9]:
neg_df_path = "data/bible_with_neg.csv"

df_with_negatives = get_negatives(df, embeddings, min_score=0.80, max_score=0.98)
df_with_negatives.to_csv(neg_df_path)



100%|██████████| 13789/13789 [09:15<00:00, 24.84it/s]


In [10]:
df_with_negatives.head()

,Unnamed: 0,chapter,start_verse,end_verse,text_ru,text_lez,book_name_ru,book_name_lez,negative_id
0,0,1,1,1,В первый день второго месяца ( в середине весн...,"Кьвед лагьай вацран сад лагьай юкъуз, исраилви...",Числа,Кьурумда,"[6, 11, 14, 38, 46, 53, 73, 75, 81, 85]"
1,1,1,45,45,"Все исроильтяне от двадцати лет и старше, годн...","Аскервиле къуллугъ авуниз виже къведай, чпин к...",Числа,Кьурумда,"[4, 5, 7, 8, 9, 12, 15, 20, 22, 23]"
2,2,1,25,25,В роду Гада их было 45 650.,Гадан тайифада абурун кьадар 45650 тир.,Числа,Кьурумда,"[3, 4, 8, 9, 10, 12, 13, 16, 21, 23]"
3,3,1,46,46,Всего их было 603 550.,Абурун виридан кьадар 603550 тир.,Числа,Кьурумда,"[2, 4, 5, 8, 9, 10, 16, 21, 24, 25]"
4,4,1,24,24,Из потомков Гада: все мужчины двадцати лет и с...,Гадан эвледрикай: аскервиле къуллугъ авуниз ви...,Числа,Кьурумда,"[1, 2, 3, 5, 7, 8, 9, 10, 12, 13]"


: 